In [1]:
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE" # on NERSC filelocking is not allowed
import h5py
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
import pickle as pkl

import tensorflow as tf
# Make notebook run on other GPUS. GPT's solution ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[1], 'GPU')  # change 1 to 0, 2, 3 as needed
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

import sys
# Path to dir model.py lives in -------
# NOTE: This needs to be modified to where your repo lives, path to /repo/path/VAE_FS/models/
# If the jupyter notebook kernel is running from VAE_FS/models/ the
# line below is not needed
sys.path.append('/global/homes/j/jananinf/projs/VAE_FS/models/')

# import the custom models and functions
from models import Qmake_encoder_set_weights, Qmake_decoder_set_weights, Qmake_discriminator, VAE_GAN_Model
from data_and_eval_utils import load_preprocessed_snl

# in gan1. We train the VAE_GAN model

2025-07-22 16:11:06.792654: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-22 16:11:08.168701: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load data
home_path = "/global/cfs/cdirs/m2616/jananinf/projsIO/VAE_FS/" # Updated to NERSC
# file_path = home_path + "preprocessed_SNL_data.h5"
# with h5py.File(file_path, 'r') as hf:           # Shapes:
#     X_train = hf['X_train'][:]                  # (3200000, 57)
#     X_test  = hf['X_test'][:]                   # (800000,  57)
#     Ato4l_data  = hf['Ato4l_data'][:]           # (55969,   57) Signal data 
#     hToTauTau_data  = hf['hToTauTau_data'][:]   # (691283,  57) ""
#     hChToTauNu_data  = hf['hChToTauNu_data'][:] # (760272,  57) ""
#     leptoquark_data = hf['leptoquark_data'][:]  # (340544,  57) ""
#     print("Data loaded from preprocessed_SNL_data.h5")

data = load_preprocessed_snl()
X_train = data['X_train']

Data loaded from preprocessed_SNL_data.h5


In [4]:
INPUT_SZ = 57
H1_SZ = 32
H2_SZ = 16
LATENT_SZ = 3
NUM_EPOCHS = 100
BATCH_SIZE = 1024
STOP_PATIENCE = 50
LR_PATIENCE = 25
DISC_H1_SZ = 8 # Size of first hidden layer of discriminator  # 8, 2 is on ATLAS-VAE-GAN
DISC_H2_SZ = 2 # "" second hidden layer ""


### Simple training loop. No parameter sweeping

In [ ]:
train = True
NUM_TRAIN = 10 
save = True
SAVE_PATH = home_path+f"/GAN_trainings/attempt7/" #
# Next attempt should go to 2
# Attempt History. The original code for each folder should also be tied to the commits. 
# 0: First attempt. GAN as copied from other repo https://github.com/max-cohen54/AD_trigger_training/blob/main/L1AD/software/VAE_GAN/L1_VAE_Analyzer_FDL_GAN_ALT.ipynb
# 1: Added GAN loss to 
# 2: Various parametric sweeps
# 3: Better file naming convention and varied clipnorm
# Notes: Smaller clipnorm ~ 0.1 tended to bring down the losses rather than blowing up.
# Keeping clipnorm to 0.1 in future trainings
# 4: Varying gamma maxes
# 5: gamma = 0 sanity check
# 6: Changed Discriminator to 8, 2 for hidden layers as in https://github.com/max-cohen54/AD_trigger_training/blob/main/L1AD/software/VAE_GAN/L1_VAE_Analyzer_FDL_GAN_ALT.ipynb
## No longer sweeping for now.
# 7: Learning rate set to 0.0001
# 8: Changed gamma linear schedule to actually stop at max gamma. previously it went up to 100 times greater than max gamma. 

early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=LR_PATIENCE, verbose=1)

for i in range(NUM_TRAIN):
    if train:
        tf.keras.backend.clear_session()

        print(f"TRAINING ITERATION {i} ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
        enc = Qmake_encoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
        dec = Qmake_decoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
        disc = Qmake_discriminator(INPUT_SZ, DISC_H1_SZ, DISC_H2_SZ)

        steps_per_epoch = X_train.shape[0] // BATCH_SIZE
        
        # Modified these setting to match atlas VAE gan repo
        vae = VAE_GAN_Model(enc, dec, disc, cycle_length=20, min_beta=0, max_beta=1, min_gamma=1, max_gamma=50)
        opt = keras.optimizers.Adam(learning_rate=0.0001)
        # --
        vae.compile(optimizer=opt)
        history = vae.fit(x=X_train, validation_split=0.2, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stopping,reduce_lr], shuffle=True)

        
        # Iterative training. 
        save_path = SAVE_PATH+f"n_{i}/" 
        if save:
            print(f"SAVING ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n")
            vae.save_weights(filepath=save_path, save_format='tf')

            # Now save the histories
            with open(save_path + f"training_history.pkl", 'wb') as f:
                pkl.dump(history.history, f)

TRAINING ITERATION 0 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~



2025-07-22 16:12:01.986966: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38366 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:41:00.0, compute capability: 8.0


Epoch 1/100


2025-07-22 16:12:04.866467: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2025-07-22 16:12:05.775847: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x5560de729a20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-07-22 16:12:05.775867: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2025-07-22 16:12:05.818012: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-07-22 16:12:06.004979: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8901
2025-07-22 16:12:06.249509: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the pro

2500/2500 [==============================] - 16s 4ms/step - loss: 14.5306 - reco_loss: 2.5644 - kl_loss: 4.1223 - disc_loss: 0.4466 - beta: 0.9900 - raw_loss: 4.0641 - w_kl_loss: 1.1099 - w_disc_loss: 24.4377 - gamma: 62.2500 - val_loss: 135.9002 - val_reco_loss: 62.4710 - val_kl_loss: 15.3244 - val_raw_loss: 77.7954 - val_disc_loss: 0.4719 - val_w_kl_loss: 15.1711 - val_w_disc_loss: 58.2581 - val_gamma: 123.4510 - val_beta: 0.9900 - lr: 1.0000e-04
Epoch 2/100
2500/2500 [==============================] - 11s 4ms/step - loss: 131.0623 - reco_loss: 8.5728 - kl_loss: 4.1979 - disc_loss: 0.7339 - beta: 1.0000 - raw_loss: 16.0968 - w_kl_loss: 5.7104 - w_disc_loss: 137.5611 - gamma: 184.7500 - val_loss: 192.4345 - val_reco_loss: 3.8718 - val_kl_loss: 0.9293 - val_raw_loss: 4.8011 - val_disc_loss: 0.7629 - val_w_kl_loss: 0.9293 - val_w_disc_loss: 187.6334 - val_gamma: 245.9510 - val_beta: 1.0000 - lr: 1.0000e-04
Epoch 3/100
2500/2500 [==============================] - 11s 4ms/step - loss: 164

TypeError: cannot pickle 'weakref' object

Moving over to large parametric sweep to find something that will work

In [ ]:
from matplotlib import pyplot as plt
# NUM_TRAIN = 4 # Train just once for now
SAVE_PATH = home_path+f"GAN_trainings/attempt6/" #
train = False
save = True
NUM_TRAIN = 3 # Train just once for now
# Next attempt should go to 2
# Attempt History. The original code for each folder should also be tied to the commits. 
# 0: First attempt. GAN as copied from other repo https://github.com/max-cohen54/AD_trigger_training/blob/main/L1AD/software/VAE_GAN/L1_VAE_Analyzer_FDL_GAN_ALT.ipynb
# 1: Added GAN loss to 
# 2: Various parametric sweeps
# 3: Better file naming convention and varied clipnorm
# Notes: Smaller clipnorm ~ 0.1 tended to bring down the losses rather than blowing up.
# Keeping clipnorm to 0.1 in future trainings
# 4: Varying gamma maxes
# 5: gamma = 0 sanity check
# 6: Changed Discriminator to 8, 2 for hidden layers as in https://github.com/max-cohen54/AD_trigger_training/blob/main/L1AD/software/VAE_GAN/L1_VAE_Analyzer_FDL_GAN_ALT.ipynb
## No longer sweeping for now.


early_stopping = EarlyStopping(patience=STOP_PATIENCE, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=LR_PATIENCE, verbose=1)

parameters = [0]
parameters_key = "max_gamma"
SAVE_PATH = SAVE_PATH + parameters_key + "/"

n_train = 3 # Train at least 3 models per parameter

if train:
    for param in parameters:
        for i in range(n_train):
            save_path = SAVE_PATH + f"{parameters_key}_{param}/"

            # Manually make the directories and file. Python can do it, but its cleaner to do it manually
            with open(SAVE_PATH +"out.txt", "a") as f:
                print(f"Variant: {parameters_key} = {param} TRAINING ITERATION {i} ~~~~~~~~~~~\n", file=f)


            tf.keras.backend.clear_session()

            enc = Qmake_encoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
            dec = Qmake_decoder_set_weights(INPUT_SZ, H1_SZ, H2_SZ, LATENT_SZ)
            disc = Qmake_discriminator(INPUT_SZ, 8, 2) # Testing out these values for now

            steps_per_epoch = X_train.shape[0] // BATCH_SIZE
            vae = VAE_GAN_Model(enc, dec, disc, cycle_length=20, min_beta=0, max_beta=1, min_gamma=1, max_gamma=50)
            opt = keras.optimizers.Adam(learning_rate=0.001) 
            history = vae.fit(x=X_train, validation_split=0.2, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stopping,reduce_lr], shuffle=True)

            # Make loss plot
            plt.figure(figsize=(12, 8))
            # Plot training losses
            for key, val in history.history.items():
                if key == 'lr':
                    continue
                plt.plot(val, label=key, 
                        linestyle = "dashed" if key[0:3] == 'val' else "solid") 
                
            # Customize the plot
            plt.title(f'Variant: {parameters_key} = {param} Training and Validation Losses Run: {i}')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            plt.grid(True)
            plt.semilogy()

            # # Show the plot
            # plt.show()
            
            # Iterative training. 
            # save_path = save_path + f"n_{i}/" # As of 7/8/25. Should be synced with vae0_analysis
            if save:
                iter_save_path = save_path +  f"n_{i}/"

                # Save progress to main out file
                with open(SAVE_PATH + "out.txt", "a") as f: 
                    print(f"SAVING Variant: {parameters_key} = {param} TRAINING ITERATION {i} ~~~~~~~~~~~\n Save path: {iter_save_path}\n", file=f, flush = True)

                # Save weights to iter specific folder
                vae.save_weights(filepath=iter_save_path , save_format='tf')
                # Now save the histories
                with open(iter_save_path + f"training_history.pkl", 'wb') as f:
                    pkl.dump(history.history, f)
                plt.savefig(iter_save_path + parameters_key + f"_{param}.png")




Plot Loss vs epoch history.

In [ ]:
import matplotlib.pyplot as plt
import pickle as pkl
# Assuming 'history' is the object returned by your model.fit() call

for i in range(NUM_TRAIN):
    save_path = SAVE_PATH + f"n_{i}/"
    with open(save_path + 'training_history.pkl', 'rb') as f:
        history = pkl.load(f)

    # Extract the loss values
    total_loss = history['loss']
    reco_loss = history['reco_loss']
    kl_loss = history['kl_loss']
    val_total_loss = history['val_loss']
    val_reco_loss = history['val_reco_loss']
    val_kl_loss = history['val_kl_loss']
    gamma = history['gamma']

    # Create a new figure
    plt.figure(figsize=(12, 8))

    # Plot training losses
    for key, val in history.items():
        if key == 'lr':
            continue
        plt.plot(val, label=key, 
                 linestyle = "dashed" if key[0:3] == 'val' else "solid") 
    # plt.plot(total_loss, label='Total Loss', color='blue')
    # plt.plot(reco_loss, label='Reconstruction Loss', color='green')
    # plt.plot(kl_loss, label='KL Loss', color='red')

    # plt.plot(history['beta'],label="beta")
    # plt.plot(history['gamma'], label="$\gamma$")

    # # Plot validation losses
    # plt.plot(val_total_loss, label='Val Total Loss', color='blue', linestyle='--')
    # plt.plot(val_reco_loss, label='Val Reconstruction Loss', color='green', linestyle='--')
    # plt.plot(val_kl_loss, label='Val KL Loss', color='red', linestyle='--')

    # Customize the plot
    plt.title(f'Training and Validation Losses Run: {i}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.semilogy()

    # Show the plot
    plt.show()

In [ ]:
# What do we want as a AD metric? the discriminator or latent space vars